# Proyecto: Visualización 3D de modelo de exposición

Github: [Github del proyecto](https://github.com/drodrguez/Visualizacion3D/)

## Descripción del proyecto

Este proyecto tiene como objetivo investigar e implementar metodologías novedosas de visualización de datos de exposición a distintas escalas espaciales, que permitan representar y comunicar de manera atractiva la información contenida en ellos, como ubicación, altura, densidad, materialidad, tipo de uso, edad, tipología estructural, etc.

## Datos disponibles

- Modelo de exposición: Los datos principales de este proyecto corresponde al modelo de exposición producto de la publicación "Development of national and local exposure models of residential structures in Chile". Este modelo se constituye de 1 archivo .txt para cada región, que incluye un catastro a nivel de bloque censal para 18 tipologías distintas de estructuras. 

- Datos de salud pública: Se obtienen directamente desde el [MINSAL](https://deis.minsal.cl/). Estos datos presentan el desafío de que son actualizados constantemente, por lo que su url es variable. Se debe pensar en como solucionar este problema.

- Datos de educación: Obtenidos desde la página de datos abiertos del [MINEDUC](http://datosabiertos.mineduc.cl/). Estos presentan el desafío de que su formato es en .rar, formato que es dificil de abordar desde la automatización, pues WinRar es un producto de licencia y no open-source.

- Datos de cartografía censal: Obtenidos desde la página del [censo](http://www.censo2017.cl/servicio-de-mapas/). Presenta el mismo problema: vienen en formato .rar.

- Datos viales: PENDIENTE


## Carga de los datos.

### Modelo de exposición. 

Los modelos de exposición se encuentran en formato .txt, y deben ser transformados a un formato adecuado para el trabajo de datos geoespaciales. Esto se realiza a través de pandas y los métodos implementados en geopandas, que nos permiten obtener la geometría directamente desde un dataset.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datashader as ds

In [2]:
# Obtenemos los path de los modelos de exposición por región almacenados en el github del proyecto.

exposure_paths = ["https://raw.githubusercontent.com/drodrguez/Visualizacion3D/main/Datos/Exposure_model/Exposure%20Model%20Block%20Reg%20"+str(num)+"_opt.txt" for num in range(1, 16) if num!= 13]
capital_exposure = ["https://raw.githubusercontent.com/drodrguez/Visualizacion3D/main/Datos/Exposure_model/ExposureModelBlockReg_13_"+str(num)+"_opt.txt" for num in range(1, 4)]
exposure_paths = exposure_paths + capital_exposure

In [3]:
exposure_geodata = dict()
capital = []
for path in exposure_paths:    
    data = pd.read_csv(path, encoding='latin-1')
    reg_id = data.region_id[0]
    if reg_id == 13:
        capital.append(data)
    else:
        geodata = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.latitude, data.longitude))          
        exposure_geodata[geodata.region_id[0]] = geodata
    
capital_data = pd.concat([data for data in capital])
geodata = gpd.GeoDataFrame(capital_data, geometry=gpd.points_from_xy(capital_data.latitude, capital_data.longitude)) 
exposure_geodata[13] = geodata

Tomaremos un modelo de exposición cualquiera y analizaremos sus características.

In [4]:
exposure_geodata[13].head()

,longitude,latitude,region_id,region_name,municipality_id,municipality_name,census_block,typology,number_dwellings,number_buildings,...,replace_cost_per_building_area(LocalCurrency/m2),replace_cost_per_building(USD),replace_cost_per_building(LocalCurrency),total_replace_cost(USD),total_replace_cost(LocalCurrency),occupants_per_dwelling,occupants_per_building,total_occupants,is_urban,geometry
0,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,1,0.31,0.31,...,NaN,41646.56,NaN,13006.92,NaN,2.93,2.93,0.91,True,POINT (-33.43581 -70.63756)
1,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,2,9.01,0.11,...,NaN,2366932.21,NaN,265499.02,NaN,1.95,156.26,17.53,True,POINT (-33.43581 -70.63756)
2,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,3,48.18,0.22,...,NaN,6327080.83,NaN,1386948.00,NaN,1.95,427.67,93.75,True,POINT (-33.43581 -70.63756)
3,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,4,28.17,0.08,...,NaN,9503050.08,NaN,787276.10,NaN,1.95,661.62,54.81,True,POINT (-33.43581 -70.63756)
4,-70.63756,-33.43581,13,Region Metropolitana,13101,Santiago,13101011001002,5,0.13,0.13,...,NaN,48037.54,NaN,6374.48,NaN,3.20,3.20,0.42,True,POINT (-33.43581 -70.63756)


In [27]:
exposure_geodata[13].columns

Index(['region_id', 'municipality_id', 'census_block', 'typology',
       'number_dwellings', 'number_buildings', 'avg_dwellings_per_building',
       'avg_number_storeys', 'total_replace_cost(USD)',
       'occupants_per_dwelling', 'occupants_per_building', 'total_occupants',
       'is_urban', 'geometry'],
      dtype='object')

Seleccionemos las columnas de interés:

In [25]:
selected_cols = ['region_id', 'municipality_id', 'census_block',
                 'typology', 'number_dwellings', 'number_buildings', 
                   'avg_dwellings_per_building', 'avg_number_storeys',
                   'total_replace_cost(USD)',  'occupants_per_dwelling',
                   'occupants_per_building', 'total_occupants',
                   'is_urban', 'geometry']

for k in exposure_geodata.keys():
    exposure_geodata[k] = exposure_geodata[k][selected_cols]                   
                   
                                      

In [26]:
exposure_geodata[13].describe()

,region_id,municipality_id,census_block,typology,number_dwellings,number_buildings,avg_dwellings_per_building,avg_number_storeys,total_replace_cost(USD),occupants_per_dwelling,occupants_per_building,total_occupants
count,541989.0,541989.000000,5.419890e+05,541989.000000,541989.000000,541989.000000,541989.000000,467902.000000,5.419890e+05,541989.000000,541989.000000,541989.000000
mean,13.0,13172.426044,1.317250e+13,8.723915,3.857092,2.621032,29.247211,4.157491,1.009639e+05,3.317381,81.925327,12.660172
std,0.0,123.610325,1.236017e+11,5.330434,10.516034,6.505172,77.449957,5.658901,4.024971e+05,0.478383,210.114674,31.919286
min,13.0,13101.000000,1.310101e+13,1.000000,0.010000,0.000000,1.000000,1.000000,6.470000e+00,1.000000,1.000000,0.010000
25%,13.0,13111.000000,1.311102e+13,5.000000,0.130000,0.010000,1.000000,1.690000,9.870400e+02,2.990000,3.530000,0.410000
50%,13.0,13120.000000,1.312008e+13,7.000000,0.750000,0.230000,1.000000,1.980000,1.348897e+04,3.480000,3.740000,2.610000
75%,13.0,13131.000000,1.313105e+13,14.000000,3.790000,2.410000,27.560000,4.260000,7.585430e+04,3.690000,81.580000,12.760000
max,13.0,13605.000000,1.360599e+13,18.000000,820.140000,510.520000,660.000000,40.000000,4.311764e+07,5.250000,1846.000000,2885.290000


In [29]:
exposure_geodata[13].info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 541989 entries, 0 to 127140
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   region_id                   541989 non-null  int64   
 1   municipality_id             541989 non-null  int64   
 2   census_block                541989 non-null  int64   
 3   typology                    541989 non-null  int64   
 4   number_dwellings            541989 non-null  float64 
 5   number_buildings            541989 non-null  float64 
 6   avg_dwellings_per_building  541989 non-null  float64 
 7   avg_number_storeys          467902 non-null  float64 
 8   total_replace_cost(USD)     541989 non-null  float64 
 9   occupants_per_dwelling      541989 non-null  float64 
 10  occupants_per_building      541989 non-null  float64 
 11  total_occupants             541989 non-null  float64 
 12  is_urban                    541989 non-null  bool 

In [30]:
exposure_geodata[13].query('number_buildings == 510.52')

,region_id,municipality_id,census_block,typology,number_dwellings,number_buildings,avg_dwellings_per_building,avg_number_storeys,total_replace_cost(USD),occupants_per_dwelling,occupants_per_building,total_occupants,is_urban,geometry
14206,13,13201,13201051099999,6,510.52,510.52,1.0,1.96,7576347.0,3.69,3.69,1881.75,True,POINT (-33.59609 -70.56493)


Surgen algunas dudas:

- ¿Por qué los valores de ```number_dwellings```, ```number_buildings```, y ```occupants_per_dwelling``` adquieren esos valores tan particulares? Son decimales, ¿deberían ser aproximados? ¿Cómo corregimos esta anomalía? ¿Por qué objetos discretos están representados de forma contínua? 

De la información disponible acerca de este modelo de exposición, tenemos el archivo ```typology_codes.csv``` que nos provee más información acerca de la categoría de tipología. Veamos este archivo.


In [31]:
typology = pd.read_csv('https://raw.githubusercontent.com/drodrguez/Visualizacion3D/main/Datos/typology/typology_codes.csv', sep=';')
typology.head()

,N,Category,Material,Description,Stories,Typology
0,1,RC structures,RC,RC house,<3,CR/LWAL/HBET:1-3/RES+RES1
1,2,RC structures,RC,low-rise RC buildings,3-9,CR/LWAL/HBET:3-9/RES+RES2
2,3,RC structures,RC,mid-rise RC buildings,10-24,CR/LWAL/HBET:10-24/RES+RES2
3,4,RC structures,RC,high-rise RC buildings,>24,CR/LWAL/HBET:25-40/RES+RES2
4,5,Masonry structures,UCB,unreinforced clay brick house,1-2,MUR+CLBRS+MOC/LWAL/HBET:1-2/RES+RES1


In [32]:
typology.set_index('N', inplace=True)
typology

,Category,Material,Description,Stories,Typology
N,,,,,
1,RC structures,RC,RC house,<3,CR/LWAL/HBET:1-3/RES+RES1
2,RC structures,RC,low-rise RC buildings,3-9,CR/LWAL/HBET:3-9/RES+RES2
3,RC structures,RC,mid-rise RC buildings,10-24,CR/LWAL/HBET:10-24/RES+RES2
4,RC structures,RC,high-rise RC buildings,>24,CR/LWAL/HBET:25-40/RES+RES2
5,Masonry structures,UCB,unreinforced clay brick house,1-2,MUR+CLBRS+MOC/LWAL/HBET:1-2/RES+RES1
6,Masonry structures,RCB,reinforced clay brick house,1-2,MR+CLBRH+RS+MOC/LWAL/HBET:1-2/RES+RES1
7,Masonry structures,CCB,conflined clay brick house,1-2,MCF+CLBRS+MOC/LWAL/HBET:1-2/RES+RES1(27%)_and_...
8,Masonry structures,CB,reinforced or confined concrete block house,1-2,MCF+CBH+MOC/LWAL/HBET:1-2/RES+RES1(73%)_and_MR...
9,Masonry structures,RCB,reinforced clay brick building,3,MR+CLBRH+RS+MOC/LWAL/HEX:3/RES+RES2


De esta información, mantendremos: 

- Stories: nos permite saber el número de pisos asociada a cada tipología.

- Material: nos permite conocer el material; esta información puede ser clave a la hora de pensar en las componentes de la visualización final.

- Category: nos provee de una variable categórica para nuestros datos.

In [23]:
selected_cols = ['Stories', 'Material', 'Category']
typology = typology[selected_cols]
typology

,Stories,Material,Category
N,,,
1,<3,RC,RC structures
2,3-9,RC,RC structures
3,10-24,RC,RC structures
4,>24,RC,RC structures
5,1-2,UCB,Masonry structures
6,1-2,RCB,Masonry structures
7,1-2,CCB,Masonry structures
8,1-2,CB,Masonry structures
9,3,RCB,Masonry structures


Generaremos distintos filtros para el modelo de exposición:

### Establecimientos de salud

In [7]:
link_salud = 'https://repositoriodeis.minsal.cl/DatosAbiertos/Establecimientos_ChileDEIS_MINSAL%2005-08-2022.xlsx'

In [8]:
salud_data = pd.read_excel(link_salud, sheet_name='Establecimientos Vig')

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell U4348 is marked as a date but the serial value 973784866 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [9]:
salud_data.head()

,Versión 20220805,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,Código Antiguo,Código Vigente,Código Madre,Código Nuevo Madre,Código Región,Nombre Región,Código Dependencia Jerárquica (SEREMI / Servic...,Nombre Dependencia Jerárquica (SEREMI / Servic...,Pertenencia al SNSS,Tipo Establecimiento,...,LATITUD [Grados decimales],LONGITUD [Grados decimales],Tipo de Prestador Sistema de Salud,Estado de Funcionamiento,Nivel de Complejidad,Modalidad de Atención,Fecha de Incorporación a la base o cambios,NaN,NaN,NaN
1,26-704,126704,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,-54.935209,-67.600393,Público,Vigente en operación,Baja Complejidad,Atención Cerrada-Hospitalaria,NaN,NaN,NaN,NaN
2,26-204,126204,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,-54.934374,-67.608895,Pendiente,Vigente en operación,Mediana Complejidad,Atención Cerrada-Hospitalaria,NaN,NaN,NaN,NaN
3,26-412,126412,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Posta de Salud Rural (PSR),...,-53.640546,-69.645848,Público,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN,NaN,NaN,NaN
4,26-414,126414,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Posta de Salud Rural (PSR),...,-53.404244,-70.990158,Público,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN,NaN,NaN,NaN


In [10]:
salud_data.columns = salud_data.iloc[0,:]
salud_data = salud_data.iloc[1:, :]
salud_data.head()

,Código Antiguo,Código Vigente,Código Madre,Código Nuevo Madre,Código Región,Nombre Región,Código Dependencia Jerárquica (SEREMI / Servicio de Salud),Nombre Dependencia Jerárquica (SEREMI / Servicio de Salud),Pertenencia al SNSS,Tipo Establecimiento,...,LATITUD [Grados decimales],LONGITUD [Grados decimales],Tipo de Prestador Sistema de Salud,Estado de Funcionamiento,Nivel de Complejidad,Modalidad de Atención,Fecha de Incorporación a la base o cambios,NaN,NaN,NaN
1,26-704,126704,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,-54.935209,-67.600393,Público,Vigente en operación,Baja Complejidad,Atención Cerrada-Hospitalaria,NaN,NaN,NaN,NaN
2,26-204,126204,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,-54.934374,-67.608895,Pendiente,Vigente en operación,Mediana Complejidad,Atención Cerrada-Hospitalaria,NaN,NaN,NaN,NaN
3,26-412,126412,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Posta de Salud Rural (PSR),...,-53.640546,-69.645848,Público,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN,NaN,NaN,NaN
4,26-414,126414,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Posta de Salud Rural (PSR),...,-53.404244,-70.990158,Público,Vigente en operación,Baja Complejidad,Atención Abierta-Ambulatoria,NaN,NaN,NaN,NaN
5,26-102,126102,No Aplica,No Aplica,12,Región De Magallanes y de la Antártica Chilena,26,Servicio de Salud Magallanes,Perteneciente,Hospital,...,-53.298162,-70.358384,Público,Vigente en operación,Baja Complejidad,Atención Cerrada-Hospitalaria,NaN,NaN,NaN,NaN


Recordemos que esta base de datos es variable, se actualiza constantemente y no tenemos la seguridad de que las columnas, tanto en su distribución como en nombre, vayan a conservar su formato para actualizaciones futuras. Luego, se hace necesario:

- 1. Reducir las columnas para conservar sólo la información necesaria para el proyecto.
- 2. Automatizar la búsqueda de estas columnas de utilidad, mediante palabras clave.

¿Qué información nos es de utilidad, entonces?

- Datos de geolocalización: latitud y longitud.

- Tipo de establecimiento: esta base de datos cuenta con diversos tipos de establecimiento: hospitales, sapu, hasta incluso centros de cirugía estética. 

- Nivel de complejidad: En el contexto del proyecto, tiene lógica asumir que, si existe compromiso de los establecimientos de mayor complejidad, el acceso a la salud, sobretodo en caso de desastres, se ve más comprometida v/s un establecimiento de baja complejidad. 

Para esto haremos un sondeo de los valores que pueden tomar estas columnas, para filtrar según lo requerido.

In [11]:
# Busqueda automática

# Latitud y longitud

def busqueda_geo(data, debug=False):
    error = 0
    error_cols = []
    eureka = dict()
    cols = list(data.columns)
    latitud_keys = ['lat', 'latitud']
    longitud_keys = ['lon', 'longitud']
    keys = latitud_keys + longitud_keys

    for col in cols:    
        try:
            for key in keys:            
                if key in col.lower():
                    for x in data[col]:
                        if type(x) == float:
                            if key in latitud_keys:
                                eureka[col] = 'latitud'
                            else:
                                eureka[col] = 'longitud'
                        break
        except:
            error += 1
            error_cols.append(col)
            pass
        
    if len(list(eureka.keys())) == 2:        
        print(f'Busqueda de datos de geolocalización terminada con {error} errores')
    else:
        print(f'No hay suficientes coincidencias')
        
    if debug:
        return error_cols
    
    return eureka



Problema de este código: si por casualidad el primer valor es un dato faltante entonces no se encontrará la columna buscada.

In [12]:
eureka = busqueda_geo(salud_data)
eureka

Busqueda de datos de geolocalización terminada con 3 errores


{'LATITUD      [Grados decimales]': 'latitud',
 'LONGITUD [Grados decimales]': 'longitud'}